In [1]:
import igl
import scipy as sp
import numpy as np
from meshplot import plot, subplot, interact
# %matplotlib widget


In [2]:
v,t, f = igl.read_mesh('./bunny.mesh')

In [3]:
plot(v,f)

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-0.016860…

In [4]:
v[:,2].min(), v[:,2].max(), 

(-0.0619614, 0.0588308)

In [5]:
slice_z = 0.6
plane = np.array([0,0,1,-((1-slice_z)*v[:,2].min()+slice_z*v[:,2].max())])
# plane = np.array([0,0,1,-((1-slice_z)*(-0.5)+slice_z*1.)])

plane_pts = v[:,0] * plane[0] + v[:,1]*plane[1] + v[:,2]*plane[2] + plane[3]

# sv, sf, j, bc = igl.marching_tets(vv, tt, vv[:,0], isovalue)

In [6]:
plane_pts.shape

(5433,)

In [7]:
isovalue = np.mean(v[:,0])

In [8]:
sv, sf, j, bc = igl.marching_tets(v, t, plane_pts, isovalue)

In [9]:
p = plot(sv,sf)
p.add_mesh(v,f)

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-0.024296…

1

In [10]:
# p = plot(sv, sf, d[0])
# p.add_mesh(v,f)
# d_ = plot(sv, sf, d[0])


In [11]:
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

In [12]:
def plot_sdf(z):
    slice_z = z
    plane = np.array([0,0,1,-((1-slice_z)*v[:,2].min()+slice_z*v[:,2].max())])
    plane_pts =v[:,0] * plane[0] + v[:,1]*plane[1] + v[:,2]*plane[2] + plane[3]
    isovalue = np.mean(v[:,0])
    sv, sf, j, bc = igl.marching_tets(v, t, plane_pts, isovalue)
    d = igl.signed_distance(sv, v, f)
    p = plot(sv, sf, d[0])
    p.add_mesh(v,f)
    plot(sv, sf, d[0])

In [13]:
interact(plot_sdf, z=0.5)

interactive(children=(FloatSlider(value=0.5, description='z', max=1.5, min=-0.5), Output()), _dom_classes=('wi…

<function __main__.plot_sdf(z)>

### Create and view plane

In [14]:
V = np.array([
    [0., 0, 0],
    [-0.1, 0, 0],
    [-.1, .2, 0],
    [0, .2, 0]
])

F = np.array([
    [0, 1, 3],
    [1, 3, 2]
])

p = plot(V, F)
p.add_mesh(v,f)

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-0.050000…

1

## Compute bounding box

In [15]:
v1 = 0
v2 = 1
v3 = 2
v4 = 3
v5 = 4
v6 = 5
v7 = 6
v8 = 7

In [16]:
bbox_tri = np.array([[v1, v2, v5], [v2, v5, v6], [v2, v3, v6], \
                [v6, v7, v3], [v3, v7, v8], [v8, v4, v3], [v4, v8, v5],\
               [v5, v4, v1], [v5, v6, v8], [v8, v6, v7], [v2, v4, v1], [v2, v4, v3]
               ])

In [17]:
zmin = v[:,-1].min() + 0.1 * v[:,-1].min()
zmax = v[:,-1].max() + 0.1 * v[:,-1].max()
ymin = v[:,-2].min() + 0.1 * v[:,-2].min() 
ymax = v[:, -2].max() + 0.1 * v[:, -2].max() 
xmin = v[:, -3].min() + 0.1 * v[:, -3].min() 
xmax = v[:, -3].max() + v[:, -3].max()
 

In [18]:
bbox_verts = np.array([[xmin, ymin, zmax],[xmax,ymin,zmax],[xmax, ymin, zmin],\
              [xmin, ymin, zmin], [xmin, ymax, zmax], [xmax, ymax, zmax],\
              [xmax, ymax, zmin], [xmin, ymax, zmin]])

In [48]:
m = np.min(v, axis=0) - 0.1 * (np.max(v, axis=0) - np.min(v, axis=0))
ma = np.max(v, axis=0) + 0.1 * (np.max(v, axis=0) - np.min(v, axis=0))

# Corners of the bounding box
v_box = np.array([[m[0], m[1], m[2]], [ma[0], m[1], m[2]], [ma[0], ma[1], m[2]], [m[0], ma[1], m[2]],
                  [m[0], m[1], ma[2]], [ma[0], m[1], ma[2]], [ma[0], ma[1], ma[2]], [m[0], ma[1], ma[2]]])

# Edges of the bounding box
f_box = np.array([[0, 1], [1, 2], [2, 3], [3, 0], [4, 5], [5, 6], [6, 7], 
                  [7, 4], [0, 4], [1, 5], [2, 6], [7, 3]], dtype=np.int)

In [49]:
# p = plot(bbox_verts, bbox_tri)
p = plot(v,f)
p.add_edges(v_box, f_box, shading={'line_color':'red'})

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-0.016860…

1

### Tetrahedralisation

![teta cow](http://groups.csail.mit.edu/graphics/classes/6.838/F01/lectures/PolygonTriangulation/tetrahedralization.gif)
![tetrahedra](https://www.mathsisfun.com/geometry/images/tetrahedron-cube.svg)

In [21]:
v_extended = np.concatenate([v, bbox_verts], 0)

In [22]:
f_extended = np.concatenate([f, bbox_tri+len(v)], 0)

In [23]:
f_extended.shape,v_extended.shape

((6978, 3), (5441, 3))

In [24]:
plot(v_extended, f_extended)

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0089205…

In [25]:
import trimesh

In [26]:
mesh = trimesh.Trimesh(v, f)

In [27]:
# mesh.export('./bunny_bounded.obj')

In [28]:
plot(v_extended, f_extended)

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0089205…

In [29]:
plot(bbox_verts, bbox_tri)

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0089205…

In [30]:
new_v = np.concatenate([bbox_verts, v], 0)

In [31]:
new_f = np.concatenate([bbox_tri, f], 0)

In [32]:
new_f_ = np.zeros_like(new_f)
new_f_[len(bbox_tri):] = f

In [33]:
new_f_[:len(bbox_tri),0] = bbox_tri[:,0]
new_f_[:len(bbox_tri),2] = bbox_tri[:,2]
new_f_[:len(bbox_tri),1] = bbox_tri[:,1]


In [34]:
new_f.shape

(6978, 3)

In [50]:

v_new = np.concatenate([bbox_verts, v], axis=0)
f_new = np.concatenate([bbox_tri, f+len(bbox_verts)], axis=0)


In [52]:
import tetgen

tet = tetgen.TetGen(v_extended, f_extended)


Non-Manifold
![nonmanifold](https://lh3.googleusercontent.com/proxy/gS-vkxj4oQcidwtIGjU3T2rRTBfTeP6RAOv8wKuDFzCI3HAd5zRGf0j-mevk2XzOHQJwliN_0W__tc383Y53RV6xfYy3)

In [36]:
tet.make_manifold()


In [37]:
vv, tt = tet.tetrahedralize(order=1, mindihedral=20.0, minratio=1, steinerleft=1000)


In [38]:
grid = tet.grid


In [39]:
tt.shape,vv.shape

((16691, 4), (4485, 3))

In [40]:
grid

UnstructuredGrid,Information
N Cells,16691
N Points,4485
X Bounds,"-9.476e-02, 6.104e-02"
Y Bounds,"3.299e-02, 1.874e-01"
Z Bounds,"-6.196e-02, 5.883e-02"
N Arrays,0


In [41]:
grid.plot()

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

In [53]:
import pyvista as pv

In [54]:
pv.set_plot_theme('document')

In [55]:
bbox_center = np.array([(bbox_verts[:,0].min() + bbox_verts[:,0].max())/2, (bbox_verts[:,1].min() + bbox_verts[:,1].max())/2, (bbox_verts[:,2].min() + bbox_verts[:,2].max())/2])

In [56]:
import pyvista as pv

In [46]:
cells = grid.cells.reshape(-1, 5)[:, 1:]
cell_center = grid.points[cells].mean(1)  
print(cell_center.shape)
# extract cells below the 0 xy plane
mask = cell_center[:, 0] < bbox_center[0] 
cell_ind = mask.nonzero()[0]
subgrid = grid.extract_cells(cell_ind)
plotter = pv.Plotter()
plotter.add_mesh(subgrid, 'lightgrey', lighting=True, show_edges=True)
# plotter.add_mesh(sphere, 'r', 'wireframe')
plotter.add_legend([[' Input Mesh ', 'r'],
                    [' Tesselated Mesh ', 'black']])
plotter.show()

(16691, 3)


ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

In [51]:
# selected_tets = tt[cell_ind]

In [54]:
# selected_verts = []
# selected_colors = []
# for t_ in selected_tets:
#     for vert in t_:
#         selected_verts.append(vv[vert])
#         selected_colors.append(d[0][vert])

In [55]:
# selected_verts = np.array(selected_verts)
# selected_colors = np.array(selected_colors)

In [56]:
# d[0].shape, vv.shape

In [57]:
# selected_verts.shape

In [47]:
cells = grid.cells.reshape(-1, 5)[:, 1:]
cell_center = grid.points[cells].mean(1)

# extract cells below the 0 xy plane
mask = cell_center[:, 2] < 0
cell_ind = mask.nonzero()[0]
selected_tets = tt[cell_ind]
subgrid = grid.extract_cells(cell_ind)

# advanced plotting
# s_col = selected_colors[cell_ind]
plotter = pv.Plotter()
plotter.add_mesh(subgrid,   show_edges=True)
# plotter.add_mesh(sphere, 'r', 'wireframe')
plotter.add_legend([[' Input Mesh ', 'r'],
                    [' Tesselated Mesh ', 'black']])
plotter.show()

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)